# Lab2 - Data Collection and Pre-processing

## Data Requirements
1. Purpose: Primary transactions file
    -  Source: Any public e-commerce sample with ≥ 500 rows (e.g. the “1000 Sales Records” CSV on ExcelBIAnalytics¹ → keep first 500) OR the 500-row synthetic file created in class
    - Notes: Must contain: date, customer_id, product, price, quantity, coupon_code (or promo field), shipping_city
    - Reference source: https://excelbianalytics.com/wp/downloads-18-sample-csv-files-data-sets-for-testing-sales/
    - Data source: Using some item of Reference source to generate primary_dataset.csv
2. Purpose: Secondary metadata file
    - Source: A second open data source of your choice (product catalogue, city look-ups, coupon descriptions, etc.)
    - Notes: You will mine this file to build your Data Dictionary and (optionally) enhance features
    - Data source: Using some item of Reference source to generate secondary_dataset.csv


## How to Generate coupon code in primary_dataset.csv

In [98]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('Original_Dataset/primary_dataset.csv')

import random
import string

# Generate coupon code
def generate_coupon_code():
    return ''.join(random.choices(string.ascii_uppercase + string.digits, k=8))

# Assign a coupon code to about 30% of rows, leave the rest as empty
df['Coupon Code'] = [generate_coupon_code() if random.random() < 0.3 else '' for _ in range(len(df))]

# Save new CSV file with the coupon codes
df.to_csv('Original_Dataset/primary_dataset_with_CouponCode.csv', index=False)

## Step 1: Hello, Data!
Load primary_dataset_with_CouponCode.csv, display first 3 rows

In [87]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('Original_Dataset/primary_dataset_with_CouponCode.csv')

# Display first 3 rows
df.head(3)

,Order Date,Order ID,Product,Price,Quantity,Total Revenue,Shipping City,Coupon Code
0,10/18/2014,686800706,Cosmetics,437.20,8446,3692591.20,Medicine Hat,NaN
1,11/7/2011,185941302,Vegetables,154.06,3018,464953.08,Kingston,P5XUQA0Z
2,10/31/2016,246222341,Baby Food,255.28,1517,387259.76,Sudbury,NaN


## Step 2: Pick the Right Container
Depiction: Justify dict vs namedtuple vs class (1–2 sentences)

Reason:
I chose to use the class because the class can be flexible and easily expand a column, that convenient for future expansion and maintenance.

## Step 3: Transaction Class and OO data structure
Implement Transaction class and use it t populate an object-oriented data structure

In [88]:
# Define a Transaction class
class Transaction:
    def __init__(self, Order_Date, Order_ID, Product, Price, Quantity, Total_Revenue, Shipping_City, Coupon_Code):
        self.Order_Date = Order_Date
        self.Order_ID = Order_ID
        self.Product = Product
        self.Price = Price
        self.Quantity = Quantity
        self.Total_Revenue = Total_Revenue
        self.Shipping_City = Shipping_City
        self.Coupon_Code = Coupon_Code
    
    def __repr__(self):
        return (f"Transaction(Order_Date={self.Order_Date}, Order_ID={self.Order_ID}, Product={self.Product}, Price={self.Price}, "
                f"Quantity={self.Quantity}, Total_Revenue={self.Total_Revenue}, Shipping_City={self.Shipping_City}, Coupon_Code={self.Coupon_Code})")


# Create a list of Transaction objects
tx = Transaction(
    Order_Date = "2025-02-01",
    Order_ID = "202510017",
    Product = "Vegetables",
    Price = 100,
    Quantity = 2,
    Total_Revenue = 200,
    Shipping_City = "Kitchener",
    Coupon_Code = "SAVE10PS"
)


# Display the transaction
print("Transaction details:")
print(tx)

Transaction details:
Transaction(Order_Date=2025-02-01, Order_ID=202510017, Product=Vegetables, Price=100, Quantity=2, Total_Revenue=200, Shipping_City=Kitchener, Coupon_Code=SAVE10PS)


## Step 4: Bulk Loader
Using the primary_dataset_with_CouponCode.csv to do load_transactions() returning list ↦ type-hinted 

In [89]:
import pandas as pd
from typing import List

# Define load transactions function
def load_transactions(filename: str) -> List[Transaction]:
    df = pd.read_csv(filename)
    # create a list of Transaction objects
    transactions = [
        Transaction(
            row['Order Date'],
            row['Order ID'],
            row['Product'],
            float(row['Price']),
            int(row['Quantity']),
            float(row['Total Revenue']),
            row['Shipping City'],
            row['Coupon Code']
        )
        for _, row in df.iterrows()
    ]
    return transactions
    

# Use bulk loader, call the function to load transactions list
transactions = load_transactions('Original_Dataset/primary_dataset_with_CouponCode.csv')

# Display the first 5 transactions
print("First 5 transactions:")

for tx in transactions[:5]:
    print(tx)

First 5 transactions:
Transaction(Order_Date=10/18/2014, Order_ID=686800706, Product=Cosmetics, Price=437.2, Quantity=8446, Total_Revenue=3692591.2, Shipping_City=Medicine Hat, Coupon_Code=nan)
Transaction(Order_Date=11/7/2011, Order_ID=185941302, Product=Vegetables, Price=154.06, Quantity=3018, Total_Revenue=464953.08, Shipping_City=Kingston, Coupon_Code=P5XUQA0Z)
Transaction(Order_Date=10/31/2016, Order_ID=246222341, Product=Baby Food, Price=255.28, Quantity=1517, Total_Revenue=387259.76, Shipping_City=Sudbury, Coupon_Code=nan)
Transaction(Order_Date=4/10/2010, Order_ID=161442649, Product=Cereal, Price=205.7, Quantity=3322, Total_Revenue=683335.4, Shipping_City=Red Deer, Coupon_Code=nan)
Transaction(Order_Date=8/16/2011, Order_ID=645713555, Product=Fruits, Price=9.33, Quantity=9845, Total_Revenue=91853.85, Shipping_City=Brandon, Coupon_Code=nan)


## Step 5: Quick Profiling
(1) Calculate the Min/mean/max price 
(2) Display unique city count and cities list

In [90]:
# Get all list of Unit Prices
prices = [tx.Price for tx in transactions]

# Calculate min, mean, and max of Unit Prices
min_price = min(prices)
max_price = max(prices)
mean_price = sum(prices) / len(prices)

# Display the min, mean, and max of Unit Prices
print(f"Min Price: {min_price}")
print(f"Max Price: {max_price}")
print(f"Mean Price: {mean_price}")

# Get all list of Countrys(cities)
cities = [tx.Shipping_City for tx in transactions]
unique_cities = set(cities)
print(f"unique cities count: {len(unique_cities)}")
print(f"cities list: {unique_cities}")

Min Price: 9.33
Max Price: 668.27
Mean Price: 274.29506
unique cities count: 49
cities list: {'Wood Buffalo', 'Victoria', 'Quebec City', 'Calgary', 'Medicine Hat', 'Peterborough', 'Kamloops', 'Belleville', 'Abbotsford', 'Saint-Jean-sur-Richelieu', 'Regina', 'Vernon', 'Edmonton', 'Drummondville', 'Sarnia', 'Chatham-Kent', 'Oshawa', 'Prince George', 'London', 'Moncton', 'Winnipeg', 'Fredericton', 'Kingston', 'Windsor', 'St. Catharines', 'Kelowna', 'Barrie', 'Nanaimo', 'Red Deer', 'Vancouver', 'Montreal', 'Kitchener', 'Lethbridge', 'Toronto', 'Saskatoon', 'Saint John', 'Sherbrooke', 'Thunder Bay', 'Brantford', 'Sudbury', 'Chilliwack', 'Halifax', 'Grande Prairie', 'Brandon', 'New Westminster', 'Hamilton', 'Ottawa', 'Moose Jaw', 'Guelph'}


## Step 6: Spot the Grime 
Identify at least three dirty data cases

In [91]:
# Case 1: find missing values in the 'Coupon Code' column
count = 0
for tx in transactions:
    if tx.Coupon_Code == '' or str(tx.Coupon_Code).lower() == 'nan':
        print("missing value of Coupon Code column:", tx)
        count += 1
        # only display 5 missing values
        if count > 5:
            break

# Case 2: find inconsistent categories in 'Product' column, check inconsistent capitalization of words
Product = set(tx.Product for tx in transactions)
print(f"\nAll Product categories:", Product)
print("\n")

# Case 3: find impossible Values in Price and Total Revenue columns, if values are 0 or negative number
found = False
for tx in transactions:
    if tx.Price <= 0:
        print("Unreasonable Price:", tx)
        found = True
    if tx.Total_Revenue <= 0:
        print("Unreasonable Total_Revenue:", tx)
        found = True

if not found:
    print("No unreasonable values found in Units Sold or Unit Cost columns.")

missing value of Coupon Code column: Transaction(Order_Date=10/18/2014, Order_ID=686800706, Product=Cosmetics, Price=437.2, Quantity=8446, Total_Revenue=3692591.2, Shipping_City=Medicine Hat, Coupon_Code=nan)
missing value of Coupon Code column: Transaction(Order_Date=10/31/2016, Order_ID=246222341, Product=Baby Food, Price=255.28, Quantity=1517, Total_Revenue=387259.76, Shipping_City=Sudbury, Coupon_Code=nan)
missing value of Coupon Code column: Transaction(Order_Date=4/10/2010, Order_ID=161442649, Product=Cereal, Price=205.7, Quantity=3322, Total_Revenue=683335.4, Shipping_City=Red Deer, Coupon_Code=nan)
missing value of Coupon Code column: Transaction(Order_Date=8/16/2011, Order_ID=645713555, Product=Fruits, Price=9.33, Quantity=9845, Total_Revenue=91853.85, Shipping_City=Brandon, Coupon_Code=nan)
missing value of Coupon Code column: Transaction(Order_Date=11/24/2014, Order_ID=683458888, Product=Cereal, Price=205.7, Quantity=9528, Total_Revenue=1959909.6, Shipping_City=Drummondville

## Step 7: Cleaning Rules
Execute fixes inside clean() and show “before/after” counts

In [92]:
# Define a clean function
def clean(transactions):
    # The number of anomalies before statistical correction
    before_missing_coupon_code = sum(
        tx.Coupon_Code == '' or str(tx.Coupon_Code).lower() == 'nan'
        for tx in transactions
    )
    before_negative_price = sum(
        tx.Price <= 0 
        for tx in transactions
    )
    before_negative_total_revenue = sum(
        tx.Total_Revenue <= 0 
        for tx in transactions
    )
    before_product = set(tx.Product for tx in transactions)
    before_product_title = set(tx.Product.title() for tx in transactions)
    before_inconsistent_product = len(before_product - before_product_title)


    cleaned = []
    for tx in transactions:
        # Clean Coupon Code dataset
        Coupon_Code = tx.Coupon_Code
        if Coupon_Code == '' or str(Coupon_Code).lower() == 'nan':
            Coupon_Code = 'NONE'
        
        # Clean Price dataset
        Price = tx.Price if tx.Price > 0 else 0
        
        # Clean Total Revenue dataset
        Total_Revenue = tx.Total_Revenue if tx.Total_Revenue > 0 else 0

        # Clean Item Type dataset
        Product = tx.Product.title() # "vegetables" -> "Vegetables"
        
        # Generate a new Transaction object with cleaned data
        cleaned_tx = Transaction(
            tx.Order_Date,
            tx.Order_ID,
            Product,
            Price,
            tx.Quantity,
            Total_Revenue,
            tx.Shipping_City,
            Coupon_Code
        )
        cleaned.append(cleaned_tx)
    
    # The number of anomalies after statistical correction
    after_missing_coupon_code = sum(
        tx.Coupon_Code == '' or str(tx.Coupon_Code).lower() == 'nan'
        for tx in cleaned
    )
    after_negative_price = sum(
        tx.Price <= 0 
        for tx in cleaned
    )
    after_negative_total_revenue = sum(
        tx.Total_Revenue <= 0 
        for tx in cleaned
    )
    after_product = set(tx.Product for tx in cleaned)
    after_product_title = set(tx.Product.title() for tx in cleaned)
    after_inconsistent_product = len(after_product - after_product_title)

    # Print the number of anomalies before and after cleaning
    print(f"Missing Coupon Code before cleaning: {before_missing_coupon_code}, after cleaning: {after_missing_coupon_code}")
    print(f"Negative Price before cleaning: {before_negative_price}, after cleaning: {after_negative_price}")
    print(f"Negative Total Revenue before cleaning: {before_negative_total_revenue}, after cleaning: {after_negative_total_revenue}")
    print(f"Inconsistent Product before cleaning: {before_inconsistent_product}, after cleaning: {after_inconsistent_product}")
    
    return cleaned

# Perform cleanup and display the number of exceptions before and after
cleaned_transactions = clean(transactions)

Missing Coupon Code before cleaning: 351, after cleaning: 0
Negative Price before cleaning: 0, after cleaning: 0
Negative Total Revenue before cleaning: 0, after cleaning: 0
Inconsistent Product before cleaning: 9, after cleaning: 0


## Step 8: Transformations
Parse coupon_code ➞ numeric discount (others apply)

In [93]:
# Coupon Code transformation to discount item
def transform_with_discount(transactions):
    coupon_to_discount = {
        'NONE': 0,
        '1DIHM8JO': 0.1,
        '22EYACAF': 0.2,
        '40CXXUES': 0.3,
    }

    transformed = []
    for tx in transactions:
        code = tx.Coupon_Code
        # Default discount 5%
        discount = coupon_to_discount.get(code, 0.05)

        tx_dict = {
            'Order_Date': tx.Order_Date,
            'Order_ID': tx.Order_ID,
            'Product': tx.Product,
            'Discounted_Price': tx.Price * (1 - discount),
            'Quantity': tx.Quantity,
            'Discounted_Total_Revenue': tx.Total_Revenue * (1 - discount),
            'Shipping_City': tx.Shipping_City,
            'Coupon_Code': code,
            'Discount': discount
        }
        transformed.append(tx_dict)

    return transformed

# Run the transformation
transformed_transactions = transform_with_discount(cleaned_transactions)

# Display the first 5 transformed transactions
for tx in transformed_transactions[:5]:
    print(tx)


{'Order_Date': '10/18/2014', 'Order_ID': 686800706, 'Product': 'Cosmetics', 'Discounted_Price': 437.2, 'Quantity': 8446, 'Discounted_Total_Revenue': 3692591.2, 'Shipping_City': 'Medicine Hat', 'Coupon_Code': 'NONE', 'Discount': 0}
{'Order_Date': '11/7/2011', 'Order_ID': 185941302, 'Product': 'Vegetables', 'Discounted_Price': 146.357, 'Quantity': 3018, 'Discounted_Total_Revenue': 441705.426, 'Shipping_City': 'Kingston', 'Coupon_Code': 'P5XUQA0Z', 'Discount': 0.05}
{'Order_Date': '10/31/2016', 'Order_ID': 246222341, 'Product': 'Baby Food', 'Discounted_Price': 255.28, 'Quantity': 1517, 'Discounted_Total_Revenue': 387259.76, 'Shipping_City': 'Sudbury', 'Coupon_Code': 'NONE', 'Discount': 0}
{'Order_Date': '4/10/2010', 'Order_ID': 161442649, 'Product': 'Cereal', 'Discounted_Price': 205.7, 'Quantity': 3322, 'Discounted_Total_Revenue': 683335.4, 'Shipping_City': 'Red Deer', 'Coupon_Code': 'NONE', 'Discount': 0}
{'Order_Date': '8/16/2011', 'Order_ID': 645713555, 'Product': 'Fruits', 'Discounted

## Step 9: Feature Engineering
Add days_since_purchase feature

In [94]:
# Importing the datetime module
from datetime import datetime

# Adding Days_Since_Purchase feature
def transform_with_features(transactions):
    transformed = []
    today = datetime.today()
    for tx in transactions:
        order_date = datetime.strptime(tx['Order_Date'], "%m/%d/%Y")
        # Calculate the number of days between Order_Date and today
        days_since_purchase = (today - order_date).days

        # Order month
        order_month = order_date.month
        # Order season
        if order_month in [3, 4, 5]:
            order_season = 'Spring'
        elif order_month in [6, 7, 8]:
            order_season = 'Summer'
        elif order_month in [9, 10, 11]:
            order_season = 'Fall'
        else:
            order_season = 'Winter'

        # Create a dictionary with the transformed data
        tx_dict = dict(tx)
        tx_dict['Days_Since_Purchase'] = days_since_purchase
        tx_dict['Order_Month'] = order_month
        tx_dict['Order_Season'] = order_season

        transformed.append(tx_dict)

    return transformed

# Run the transformation
feature_transactions = transform_with_features(transformed_transactions)

# Display the first 5 transformed transactions
for tx in feature_transactions[:5]:
    print(tx)

{'Order_Date': '10/18/2014', 'Order_ID': 686800706, 'Product': 'Cosmetics', 'Discounted_Price': 437.2, 'Quantity': 8446, 'Discounted_Total_Revenue': 3692591.2, 'Shipping_City': 'Medicine Hat', 'Coupon_Code': 'NONE', 'Discount': 0, 'Days_Since_Purchase': 3874, 'Order_Month': 10, 'Order_Season': 'Fall'}
{'Order_Date': '11/7/2011', 'Order_ID': 185941302, 'Product': 'Vegetables', 'Discounted_Price': 146.357, 'Quantity': 3018, 'Discounted_Total_Revenue': 441705.426, 'Shipping_City': 'Kingston', 'Coupon_Code': 'P5XUQA0Z', 'Discount': 0.05, 'Days_Since_Purchase': 4950, 'Order_Month': 11, 'Order_Season': 'Fall'}
{'Order_Date': '10/31/2016', 'Order_ID': 246222341, 'Product': 'Baby Food', 'Discounted_Price': 255.28, 'Quantity': 1517, 'Discounted_Total_Revenue': 387259.76, 'Shipping_City': 'Sudbury', 'Coupon_Code': 'NONE', 'Discount': 0, 'Days_Since_Purchase': 3130, 'Order_Month': 10, 'Order_Season': 'Fall'}
{'Order_Date': '4/10/2010', 'Order_ID': 161442649, 'Product': 'Cereal', 'Discounted_Price

## Step 10:  Mini-Aggregation
Using dict or pandas.groupby to do revenue per shipping_city

In [ ]:
# Statistics revenue per Shipping City using python dict
def calculate_revenue_per_cit(transactions):
    revenue_by_city = {}
    for tx in transactions:
        city = tx['Shipping_City']  # Assuming 'Country' is the shipping city
        revenue = tx['Discounted_Total_Revenue']
        if city not in revenue_by_city:
            revenue_by_city[city] = 0
        revenue_by_city[city] += revenue
    
    return revenue_by_city

# Calculate revenue per city
revenue_per_city = calculate_revenue_per_cit(feature_transactions)

# Display the revenue per city
print("Statistics Revenue per Shipping City:")
for city, revenue in sorted(revenue_per_city.items()):
    print(f"{city}: {revenue:.2f}")

# Save revenue per Shipping City to a CSV file
#import pandas as pd

#df = pd.DataFrame(list(revenue_per_city.items()), columns=['City', 'Total Revenue'])
#df = df.sort_values(by='Total Revenue', ascending=False) # Sort by Total Revenue from highest to lowest
#df.to_csv('Organize_Dataset/total_revenue_by_city.csv', index=False, encoding='utf-8', float_format='%.2f')
#print("Total Revenue by city saved to 'total_revenue_by_city.csv'.")

Statistics Revenue per Shipping City:
Abbotsford: 20645214.87
Barrie: 9119996.10
Belleville: 16526917.54
Brandon: 17300731.03
Brantford: 14046608.79
Calgary: 26098957.19
Chatham-Kent: 13150893.38
Chilliwack: 11501475.66
Drummondville: 26589884.68
Edmonton: 5243855.35
Fredericton: 4591646.99
Grande Prairie: 8699784.46
Guelph: 6358982.79
Halifax: 17553729.49
Hamilton: 12256136.38
Kamloops: 19138942.45
Kelowna: 14530392.29
Kingston: 14025164.93
Kitchener: 16791994.67
Lethbridge: 17684923.27
London: 5880386.29
Medicine Hat: 16621153.60
Moncton: 35640351.90
Montreal: 4603649.23
Moose Jaw: 13497338.30
Nanaimo: 7265879.06
New Westminster: 10418812.08
Oshawa: 22561448.52
Ottawa: 5030918.39
Peterborough: 8791063.25
Prince George: 21084309.83
Quebec City: 18702403.12
Red Deer: 13072665.21
Regina: 10835347.04
Saint John: 6878318.27
Saint-Jean-sur-Richelieu: 25439820.67
Sarnia: 15579157.63
Saskatoon: 7078354.75
Sherbrooke: 14336089.06
St. Catharines: 14133550.92
Sudbury: 18095116.00
Thunder Bay: 1

## Step 11: Serialization Checkpoint
Save cleaned data(primary_dataset_with_CouponCode.csv) to JSON and Parquet

In [96]:
import pandas as pd

df = pd.DataFrame(feature_transactions)

# Round only float fields 
float_cols = df.select_dtypes(include=['float']).columns
df[float_cols] = df[float_cols].round(2)

# Save the DataFrame to a CSV file
df.to_csv('Organize_Dataset/primary_dataset_with_CouponCode_Serialization.csv', index=False, encoding='utf-8')

# Save the DataFrame to a JSON file
df.to_json('Organize_Dataset/primary_dataset_with_CouponCode_Serialization.json', orient='records', force_ascii=False)

# Save the DataFrame to a Parquet file
df.to_parquet('Organize_Dataset/primary_dataset_with_CouponCode_Serialization.parquet', index=False)

print("Serialization checkpoint complete: CSV, JSON and Parquet saved.")

# Load the JSON and Parquet files
df_jaon = pd.read_json('Organize_Dataset/primary_dataset_with_CouponCode_Serialization.json')
print(df_jaon.head(3))
print("\n")

df_parquet = pd.read_parquet('Organize_Dataset/primary_dataset_with_CouponCode_Serialization.parquet')
print(df_parquet.head(3))

Serialization checkpoint complete: CSV, JSON and Parquet saved.
   Order_Date   Order_ID     Product  Discounted_Price  Quantity  \
0  10/18/2014  686800706   Cosmetics            437.20      8446   
1   11/7/2011  185941302  Vegetables            146.36      3018   
2  10/31/2016  246222341   Baby Food            255.28      1517   

   Discounted_Total_Revenue Shipping_City Coupon_Code  Discount  \
0                3692591.20  Medicine Hat        NONE      0.00   
1                 441705.43      Kingston    P5XUQA0Z      0.05   
2                 387259.76       Sudbury        NONE      0.00   

   Days_Since_Purchase  Order_Month Order_Season  
0                 3874           10         Fall  
1                 4950           11         Fall  
2                 3130           10         Fall  


   Order_Date   Order_ID     Product  Discounted_Price  Quantity  \
0  10/18/2014  686800706   Cosmetics            437.20      8446   
1   11/7/2011  185941302  Vegetables            146.

## Step 12: Soft Interview Reflection
Explaining how OOP helped (< 120 words)

Reason: When I use Object-Oriented Programming (OOP) to write my code, I can easily clean data, transform data items, and perform data feature engineering. This can display the dataset status clearly, making it easy to identify the data information and perform further data analysis. Also, OOP can reuse code efficiently and quickly maintain my code. Thus, I can focus on some functional requirements to modify the code at any time. On the other hand, OOP can improve code readability and extensibility. This helps me organize the data and analyze the data.

## Data-Dictionary
(1) Merge field definitions from the primary_dataset_with_CouponCode.csv header and the secondary_dataset.csv metadata source.
(2) Present as a tidy Markdown table including the new columns, for example: Field, Type, Description, Source.

In [97]:
import pandas as pd

# Read the primary data fields
primary_df = pd.read_csv('Organize_Dataset/primary_dataset_with_CouponCode_Serialization.csv', nrows=0)
primary_fields = pd.DataFrame({'Field': primary_df.columns})
primary_fields['Source'] = 'Primary'
                              
# Read secondary metadata
secondary_df = pd.read_csv('Original_Dataset/secondary_dataset.csv')
# If the field name is not 'Field', please adjust
if 'Field' not in secondary_df.columns:
    # If is 'Column'
    secondary_df = secondary_df.rename(columns={'Column': 'Field'})
secondary_df['Source'] = 'Secondary'

# Merge primary and secondary fields
merged = primary_fields.merge(secondary_df, on='Field', how='left', suffixes=('', '_sec'))

#
merged['Type'] = merged['Type'].fillna('Unknown')
merged['Description'] = merged['Description'].fillna('No description')
merged['Source'] = merged['Source'].fillna('Primary')

# Output Markdown table
merged = merged[['Field', 'Type', 'Description', 'Source']]

print("| Field | Type | Description | Source |")
print("|-------|------|-------------|--------|")
for _, row in merged.iterrows():
    print(f"| {row['Field']} | {row['Type']} | {row['Description']} | {row['Source']} |")


KeyError: 'Field'